# スタックとヒープを知る

- 2018/10/11 勉強会

# アジェンダ

1. 調べようと思ったきっかけ
1. メモリ領域概観
2. スタックについて
3. ヒープについて
4. まとめ
5. 参考

# 調べようと思ったきっかけ

- スタックとヒープわかってないとRustわけわかめになる

# メモリについて

- メモリは1Byte(8bit)ごとにアドレスが振られている
- 実行ファイルにコンパイルすると変数名は16進数のアドレス値に変換される
- 1つのプロセスは1つのメモリ空間と複数のスレッドを持つ
    - 1つのスレッドは1つの連続したメモリ領域のスタックを持つ

# メモリ領域概観

- メモリ領域の「サイズ」はビルド時に決定する
- スタックとヒープに格納されるデータは動的に決まる
    - ので、実行するまでどこを使うか決まらない
    - だから、下3つ以外の部分を上下両端から使っていく

![](https://he-s3.s3.amazonaws.com/media/uploads/383f472.png)

## 5つのメモリ領域

- 置かれるアドレスが
    - 実行時に決まるものが「動的」
    - コンパイル時に決まるものが「静的」

---
### 静的な領域
- text
    - プログラムの内、機械語の部分を置く
- data
    - 初期値のあるグローバル変数を置く
- bss
    - 初期値のないグローバル変数を置く
    - Block Started by Symbolの略
    - [参考リンク](http://www.ertl.jp/~takayuki/readings/info/no03.html)
    
### 動的な領域
- heap
    - プログラムのデータを置く
    - 動的にサイズが確保されるものを置く
- stack
    - 関数の引数やローカル変数を置く


## アドレスをGoで確認してみる

#### 複数の変数を宣言してアドレスを出力する

- これらの値はスタック領域に入る

```
package main

func main() {
	a, b, c := 11, 12, 13
	println(&a) // アドレスを出力
	println(&b)
	println(&c)
}

```

#### 出力結果
```
> go run address.go
0xc000040780
0xc000040778
0xc000040770 // 8bitずつ減っている
```

# スタックについて

- 積み上げる、堆積物という意味
    - きれいに整然と積み上げていくイメージ
- LIFO

![](https://www.tutorialspoint.com/data_structures_algorithms/images/stack_representation.jpg)

- 【画像引用元】[Data Structures and Algorithms Stack](https://www.tutorialspoint.com/data_structures_algorithms/stack_algorithm.htm)

## スタックについて

- コンパイラやOSが自動で割り当てる領域
    - 領域の解放も自動的にやってくれる
    - 楽
- メモリ割り当てが高速
    - ローカル変数は全て事前にわかるので一度で割り当てられる
    - ヒープ割り当てより2~3桁速い(要出典)


## コードでの解釈

- スコープが入れ子になっているとき、内側のスコープで使うデータ領域は後から確保し、先に解放される
    
![](https://ufcpp.net/media/ufcpp2000/computer/fig/Essential/Stack.png)

- わかりやすい図解
    - [スタックメモリとヒープメモリ](http://article.higlabo.com/ja/stack_and_heap.html)
    - 【画像引用元】[メモリ管理 - コンピュータの基礎知識 | ++C++; // 未確認飛行 C](https://ufcpp.net/study/computer/MemoryManagement.html)

## 気をつけること

- スタックオーバーフロー
    - スタックのメモリのサイズを超えてpushすること
- スタックアンダーフロー
    - スタックが空の状態でさらにpopしようとすること
- スコープを抜ければスタック領域が強制的に解放される
    - ポインタがスタック領域を指すときは解放後にアクセスしないようにする
    
    


# ヒープについて

- 塊、山積みという意味
    - 乱雑に山積みするイメージ

![](https://craftofcoding.files.wordpress.com/2015/12/stackmemory4.jpg)

## ヒープについて

- 動的に割り当てたり、解放できる領域
- メモリの確保と解放を自分自身で行う必要がある
    - サイズは動的に指定できる
    - 確保、解放する順番は任意
- 低速
    - アロケート時は空いている領域を検索する
    - 読み込み時もポインタアドレスから参照する

## 用途

- データを必要とするスコープがはっきりしないとき
    - さまざまな箇所から参照される場合など
- ファイルを読み込むとき
    - ファイルサイズがわからないから

## C言語では

- malloc()でメモリアロケーション
- free()で解放

------

- 擬似コード
    - 【参考】[ヒープとスタック | 学校では教えてくれないこと | [技術コラム集]組込みの門 | ユークエスト株式会社](https://www.uquest.co.jp/embedded/learning/lecture16.html)

```
ファイルオープン;
file_sizeを取得;
buf = malloc(file_size);
bufにファイル内容を読み出し;
free(buf);
```

    


## C++では

- new演算子でメモリアロケーション
- delete演算子で解放
- コンストラクタ、デストラクタがある辺りが若干違う
    - http://book.geocities.jp/muu5muu/doc1/sample6.html
    - https://brain.cc.kogakuin.ac.jp/~kanamaru/lecture/C++2/09/09-02.html





## Rustでは

- `Box<T>型`をつかうとヒープ上にアロケートできる
    - `i32`型の値5をxに格納
    - `let x: Box<i32> = Box::new(5)`
- [他の方法](http://mmi.hatenablog.com/entry/2017/08/06/230823)

## ちなみにGoでは

- スタックとヒープを気にする必要がない
    - コンパイラがよしなに行ってくれる
    - newしても必要がなければスタックにアロケートする
    - 【参考】[golangではスタックとヒープを気にする必要が無い - Qiita](https://qiita.com/rookx/items/a1e3d057a0ed71424094)

## 気をつけること

- 確保、解放を頻繁にやりすぎるとフラグメンテーションの原因になる
    - 参照の局所性が失われ、キャッシュのヒット率低下が起こり、処理性能が悪化する
    - GC後、使用領域を一箇所にまとめるコンパクションという処理を行うこともある
- 確保に失敗することもある
    - 物理的な空きメモリがないとき
    - アドレス空間に空きがないとき

# まとめ

- メモリにはいくつかの領域がある
- スタックはLIFOでスコープが寿命
- ヒープは自分で確保と解放を行う

# 参考

- [ヒープとスタック | 学校では教えてくれないこと | [技術コラム集]組込みの門 | ユークエスト株式会社](https://www.uquest.co.jp/embedded/learning/lecture16.html)
- [スタックメモリとヒープメモリ](http://article.higlabo.com/ja/stack_and_heap.html)
- [メモリ管理 - コンピュータの基礎知識 | ++C++; // 未確認飛行 C](https://ufcpp.net/study/computer/MemoryManagement.html)
- [メモリとスタックとヒープとプログラミング言語 | κeenのHappy Hacκing Blog](https://keens.github.io/blog/2017/04/30/memoritosutakkutohi_puto/)
- [malloc ライブラリのメモリ管理構造 | 技術文書 | 技術情報 | VA Linux Systems Japan株式会社](https://www.valinux.co.jp/technologylibrary/document/linux/malloc0001/)(未読)
- [ポインタと配列 - A Day In The Life](http://glassonion.hatenablog.com/entry/20090414/1239716083)